In [35]:
!pip install catboost scikit-learn

In [37]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from catboost import Pool, CatBoostClassifier

In [4]:
df_answers = pd.read_csv('/Users/gyzavyalov/Library/Mobile Documents/com~apple~CloudDocs/!Tree/public/reviews-summarization/reviews-summarization/notebooks/data/answers_202405192130.csv')
df_aspects = pd.read_csv('/Users/gyzavyalov/Library/Mobile Documents/com~apple~CloudDocs/!Tree/public/reviews-summarization/reviews-summarization/notebooks/data/aspects_202405192134.csv')
df_reviews = pd.read_csv('/Users/gyzavyalov/Library/Mobile Documents/com~apple~CloudDocs/!Tree/public/reviews-summarization/reviews-summarization/notebooks/data/reviews_202405192134.csv')


In [15]:
aspects_map = {
  df_aspects['id'][x]: df_aspects['aspect'][x]
  for x in range(df_aspects.shape[0])
}
reviews_map = {
  df_reviews['id'][x]: df_reviews['review_body'][x]
  for x in range(df_reviews.shape[0])
}

In [31]:
aspect_state = {
  1: 'хорошо',
  0: 'плохо',
  2: 'отсутствует',
  3: 'skip',
}

answers = pd.DataFrame(columns=['review', 'aspect', 'answer'])
for ind, row in df_answers.iterrows():
  review = reviews_map.get(row.review)
  aspect = aspects_map.get(row.aspect)
  answer = aspect_state[row.answer]
  answers.loc[ind] = [review, aspect, answer]

answers.head()

,review,aspect,answer
0,Какие только безумно-замечательные идеи не при...,Эмоциональное воздействие,плохо
1,"Этот фильм, можно сказать, путеводитель по стр...",Юмор,плохо
2,На фильм я пошел благодаря высокому рейтингу К...,Визуальные эффекты,отсутствует
3,Возможно ли прожить жизнь с улыбкой на лице? С...,Саундтрек/музыка,отсутствует
4,"Подмечу важное, фильмы 'о мальчике который выж...",Саундтрек/музыка,отсутствует


In [44]:
text_features = ['review']
cat_features = ['aspect']

In [57]:
train, test = train_test_split(answers, test_size=0.2)
def process(data):
  df = data.copy()
  df = df[df['aspect'] == 'Юмор']
  return df.drop(['answer'], axis=1), df['answer']

X_train, Y_train = process(train)
X_test, Y_test = process(test)

In [58]:
X_train.head()

,review,aspect
1,"Этот фильм, можно сказать, путеводитель по стр...",Юмор
70,В детстве я как и многие смотрел мультсериал о...,Юмор
289,Перед нами именно что классическая кинокартина...,Юмор
241,Пираты Карибского моря это один из моих любимы...,Юмор
407,"Есть фильмы, а есть легенды. Последних не так ...",Юмор


In [59]:
learn_pool = Pool(
    X_train, 
    Y_train, 
    cat_features=cat_features,
    text_features=text_features,
    feature_names=list(X_train)
)
test_pool = Pool(
    X_test, 
    Y_test, 
    cat_features=cat_features,
    text_features=text_features,
    feature_names=list(X_train)
)

catboost_default_params = {
    'iterations': 10_000,
    'learning_rate': 0.01,
    'eval_metric': 'Accuracy',
}

catboost_default_params.update({})

model = CatBoostClassifier(**catboost_default_params)
model.fit(learn_pool, eval_set=test_pool, verbose=True)

0:	learn: 0.7462687	test: 0.6428571	best: 0.6428571 (0)	total: 45.4ms	remaining: 7m 33s
1:	learn: 0.6865672	test: 0.6428571	best: 0.6428571 (0)	total: 67.6ms	remaining: 5m 37s
2:	learn: 0.6716418	test: 0.6428571	best: 0.6428571 (0)	total: 112ms	remaining: 6m 14s
3:	learn: 0.6716418	test: 0.6428571	best: 0.6428571 (0)	total: 155ms	remaining: 6m 26s
4:	learn: 0.6716418	test: 0.6428571	best: 0.6428571 (0)	total: 199ms	remaining: 6m 38s
5:	learn: 0.6716418	test: 0.6428571	best: 0.6428571 (0)	total: 246ms	remaining: 6m 50s
6:	learn: 0.6716418	test: 0.6428571	best: 0.6428571 (0)	total: 289ms	remaining: 6m 53s
7:	learn: 0.6716418	test: 0.6428571	best: 0.6428571 (0)	total: 357ms	remaining: 7m 25s
8:	learn: 0.6716418	test: 0.6428571	best: 0.6428571 (0)	total: 410ms	remaining: 7m 35s
9:	learn: 0.6716418	test: 0.6428571	best: 0.6428571 (0)	total: 479ms	remaining: 7m 58s
10:	learn: 0.6716418	test: 0.6428571	best: 0.6428571 (0)	total: 533ms	remaining: 8m 4s
11:	learn: 0.6716418	test: 0.6428571	best

KeyboardInterrupt: 